In [1]:
import gcsfs
import cftime
import intake
from cmip6_preprocessing.preprocessing import combined_preprocessing
from fastjmd95 import rho
from xgcm import Grid
import numpy as np

In [2]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6-noQC.json"
col = intake.open_esm_datastore(cat_url)
cat = col.search(table_id='Omon', 
                 experiment_id=['historical'],
                 variable_id=['thetao','so'],
                 member_id=['r1i1p1f1'],
                 grid_label='gr',
                 source_id=['GFDL-ESM4', 'NorESM2-LM']
                )

In [3]:
cmip6_compiled = cat.to_dataset_dict(
            zarr_kwargs={'consolidated':True, 'decode_times':False},
            preprocess=combined_preprocessing
            )


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/Users/odyssey/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/Users/odyssey/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self

Save density as sigma_0

In [4]:
for k, ds in cmip6_compiled.items():
    ds['sigma_0'] = rho(ds.so, ds.thetao, 0) - 1000
    # use xr.set_coords?

In [5]:
print(type(cmip6_compiled))
for k, ds in cmip6_compiled.items():
    print(k)
    print(list(ds.dims))
    print(list(ds.variables))
    print(list(ds.coords))

<class 'dict'>
CMIP.NOAA-GFDL.GFDL-ESM4.historical.Omon.gr
['bnds', 'lev', 'member_id', 'time', 'vertex', 'x', 'y']
['y', 'lat_bounds', 'lev', 'lev_bounds', 'x', 'lon_bounds', 'so', 'time', 'time_bounds', 'bnds', 'lon', 'lat', 'lon_verticies', 'lat_verticies', 'vertex', 'member_id', 'thetao', 'sigma_0']
['y', 'lat_bounds', 'lev', 'lev_bounds', 'x', 'lon_bounds', 'time', 'time_bounds', 'bnds', 'lon', 'lat', 'lon_verticies', 'lat_verticies', 'vertex', 'member_id']
CMIP.NCC.NorESM2-LM.historical.Omon.gr
['bnds', 'lev', 'member_id', 'time', 'vertex', 'x', 'y']
['x', 'y', 'lat', 'lev', 'lev_bounds', 'lon', 'so', 'time', 'time_bounds', 'lat_verticies', 'lon_verticies', 'bnds', 'vertex', 'lon_bounds', 'lat_bounds', 'member_id', 'thetao', 'sigma_0']
['x', 'y', 'lat', 'lev', 'lev_bounds', 'lon', 'time', 'time_bounds', 'lat_verticies', 'lon_verticies', 'bnds', 'vertex', 'lon_bounds', 'lat_bounds', 'member_id']


Transform potential temperature and salinity

In [6]:
for k, ds in cmip6_compiled.items():
    
    grid = Grid(
            ds,
            periodic=False,
            coords={'Z':{'center':'lev', 'outer':'lev_outer'}})
    
    target_values = levels=np.arange(24,28, 0.4)
    
    ds['so_sigma'] = grid.transform(
              ds.so,
              'Z',
              target_values,
              target_data=ds.sigma_0
                )
    
    ds['thetao_sigma'] = grid.transform(
              ds.thetao,
              'Z',
              target_values,
              target_data=ds.sigma_0
                )

MergeError: unable to determine if these variables should be coordinates or not in the merged result: {'sigma_0'}